In [57]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [58]:
from google.colab import drive
drive.mount('Colab')

Drive already mounted at Colab; to attempt to forcibly remount, call drive.mount("Colab", force_remount=True).


In [59]:
data=pd.read_csv('/content/Colab/MyDrive/Colab/Flipkart refrigerator Data/cleaned Ref Data.csv')

In [60]:
data.head()

,Product Name,Product Rating,Prices in Rs,Max Price,In The Box,Door Type,Refrigerator Type,Defrosting Type,Compressor Type,Capacity,Number of Doors,Star Rating,Coolpad,Toughened Glass,Built-in Stabilizer,Company Name,Height,Width,Depth,Weight
0,Whirlpool 190 L Direct Cool Single Door 2 Star...,4.3,12740.0,14900.0,"1 Refrigerator Unit, Warranty Card, User Manual",Single Door,Top Freezer,Direct Cool,normal compressor,190,1,2.0,no,yes,Yes,Whirlpool,119.1,53.6,60.4,32.4
1,Whirlpool 190 L Direct Cool Single Door 2 Star...,4.3,11990.0,14400.0,"1 Refrigerator, User Manual, Warranty Card",Single Door,Top Freezer,Direct Cool,normal compressor,190,1,2.0,no,no,Yes,Whirlpool,119.1,53.6,60.4,32.4
2,Whirlpool 190 L Direct Cool Single Door 2 Star...,4.3,13740.0,16000.0,"1 Refrigerator Unit, Warranty Card, User Manual",Single Door,Top Freezer,Direct Cool,normal compressor,190,1,2.0,no,yes,Yes,Whirlpool,119.1,53.6,60.4,32.4
3,Whirlpool 190 L Direct Cool Single Door 2 Star...,4.3,13699.0,16150.0,1 Refrigerator\nUser Manual and Warranty Card,Single Door,Top Freezer,Direct Cool,normal compressor,190,1,2.0,yes,yes,Yes,Whirlpool,119.1,53.6,60.4,32.4
4,Whirlpool 190 L Direct Cool Single Door 2 Star...,4.3,13399.0,14950.0,"1 Refrigerator, User Manual, Warranty Card",Single Door,Top Freezer,Direct Cool,normal compressor,190,1,2.0,no,yes,Yes,Whirlpool,119.1,53.6,60.4,32.4


In [61]:
data.shape

(778, 20)

In [62]:
data.isnull().sum()

Product Name           0
Product Rating         0
Prices in Rs           0
Max Price              0
In The Box             2
Door Type              0
Refrigerator Type      0
Defrosting Type        0
Compressor Type        0
Capacity               0
Number of Doors        0
Star Rating            0
Coolpad                0
Toughened Glass        0
Built-in Stabilizer    0
Company Name           0
Height                 0
Width                  0
Depth                  0
Weight                 0
dtype: int64

In [63]:
data["In The Box"].fillna('NA', inplace=True)

In [64]:
data.isnull().sum()

Product Name           0
Product Rating         0
Prices in Rs           0
Max Price              0
In The Box             0
Door Type              0
Refrigerator Type      0
Defrosting Type        0
Compressor Type        0
Capacity               0
Number of Doors        0
Star Rating            0
Coolpad                0
Toughened Glass        0
Built-in Stabilizer    0
Company Name           0
Height                 0
Width                  0
Depth                  0
Weight                 0
dtype: int64

In [65]:
# building custom function for getting dummies of categorical column
def dummies(x,df):
    temp = pd.get_dummies(df[x],drop_first=True,prefix=x)
    df = pd.concat([df, temp], axis = 1)
    df.drop([x], axis = 1, inplace = True)
    return df
data=dummies('Door Type',data)
data=dummies('Refrigerator Type',data)
data=dummies('Defrosting Type',data)
data=dummies('Compressor Type',data)
data=dummies('Coolpad',data)
data=dummies('Toughened Glass',data)
data=dummies('Built-in Stabilizer',data)
data=dummies('Company Name',data)

In [66]:
data.head()

,Product Name,Product Rating,Prices in Rs,Max Price,In The Box,Capacity,Number of Doors,Star Rating,Height,Width,Depth,Weight,Door Type_Multi-Door,Door Type_Side by Side,Door Type_Single Door,Refrigerator Type_Not Known,Refrigerator Type_Top Freezer,Defrosting Type_Frost Free,Compressor Type_multi inverter technology,Compressor Type_normal compressor,Compressor Type_rotary compressor,Coolpad_yes,Toughened Glass_yes,Built-in Stabilizer_Yes,Company Name_CANDY,Company Name_Croma,Company Name_Electrolux,Company Name_Galanz,Company Name_Godrej,Company Name_Haier,Company Name_Hisense,Company Name_Hitachi,Company Name_LG,Company Name_Liebherr,Company Name_Lifelong,Company Name_Lloyd,Company Name_MITASHI,Company Name_MOTOROLA,Company Name_MarQ,Company Name_Midea,Company Name_ONIDA,Company Name_Panasonic,Company Name_SAMSUNG,Company Name_TOSHIBA,Company Name_Voltas,Company Name_Whirlpool
0,Whirlpool 190 L Direct Cool Single Door 2 Star...,4.3,12740.0,14900.0,"1 Refrigerator Unit, Warranty Card, User Manual",190,1,2.0,119.1,53.6,60.4,32.4,0,0,1,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,Whirlpool 190 L Direct Cool Single Door 2 Star...,4.3,11990.0,14400.0,"1 Refrigerator, User Manual, Warranty Card",190,1,2.0,119.1,53.6,60.4,32.4,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,Whirlpool 190 L Direct Cool Single Door 2 Star...,4.3,13740.0,16000.0,"1 Refrigerator Unit, Warranty Card, User Manual",190,1,2.0,119.1,53.6,60.4,32.4,0,0,1,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,Whirlpool 190 L Direct Cool Single Door 2 Star...,4.3,13699.0,16150.0,1 Refrigerator\nUser Manual and Warranty Card,190,1,2.0,119.1,53.6,60.4,32.4,0,0,1,0,1,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,Whirlpool 190 L Direct Cool Single Door 2 Star...,4.3,13399.0,14950.0,"1 Refrigerator, User Manual, Warranty Card",190,1,2.0,119.1,53.6,60.4,32.4,0,0,1,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


### `Target column is Prices `

In [67]:
y=data['Prices in Rs']
x=data.drop(['Prices in Rs','Product Name','In The Box',],axis=1)

In [68]:
from sklearn.preprocessing import StandardScaler

In [69]:
# using standard scaler
scaler = StandardScaler()
# transform data
x= scaler.fit_transform(x)

In [70]:
from sklearn.model_selection import train_test_split

In [71]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=21)

## `Linear Model`

In [72]:
from sklearn import datasets, linear_model, metrics 
from sklearn.metrics import mean_squared_error, r2_score

In [73]:
reg = linear_model.LinearRegression() 
reg.fit(x_train,y_train) 
y_pred=reg.predict(x_test)

In [74]:
print('Coefficients: \n', reg.coef_) 

Coefficients: 
 [-5.60225726e+02  1.96037464e+04  1.10004032e+03 -2.16920202e+03
 -9.12668706e+01 -1.10770187e+03 -5.99770180e+02  1.65077763e+00
  4.05530612e+03  6.29578803e+02 -1.44101919e+03 -1.83466468e+03
 -1.38289176e+02  2.68919179e+02 -2.41995754e+02 -7.87207131e+02
  2.98970686e+02 -1.79360226e+02 -3.83950310e+02 -2.49037497e+02
  8.42587965e+01 -6.41637161e+01  9.88889635e+01 -2.38742359e-12
 -8.43916851e+02  2.08661727e+02 -1.28298744e+03  5.11883721e+02
 -3.75893131e+02  9.50775528e+02 -4.72463671e+02  2.62370672e+02
 -1.18360915e+02 -2.98447968e+02 -3.08930834e+01  3.60473119e+01
 -6.87504266e+02 -3.85784552e+02  2.69805517e+02  1.26126798e+03
  1.22929422e+02 -2.88048914e+02  6.76400678e+02]


In [75]:
reg.score(x_test,y_test)

0.9522562189651956

In [76]:
r2_score(y_test,y_pred)

0.9522562189651956

In [101]:
y_pred_train_LR = reg.predict(x_train)

In [103]:
print('Training set score:',(reg.score(x_train, y_train)))

print('Test set score:',(reg.score(x_test, y_test)))

Training set score: 0.9796616311463349
Test set score: 0.9522562189651956


## `Random Forest`

In [77]:
from sklearn.ensemble import RandomForestRegressor

In [78]:
from sklearn.model_selection import  KFold, GridSearchCV, RandomizedSearchCV

In [79]:
params = [{'n_estimators': [10,20,50,70],
           'min_samples_split': [2, 5, 10],
           'min_samples_leaf': [1, 2, 5],
           'max_depth': [20, 30, 40,50]}]

In [80]:
model = RandomForestRegressor()

cv_scheme = KFold(n_splits=5, shuffle=True, random_state=1)
cv = GridSearchCV(estimator=model, param_grid=params, scoring='r2',
                  cv=cv_scheme, return_train_score=True, n_jobs=5,verbose=True)

cv.fit(x_train, y_train)

print("Best parameters for Random Forest Regression:")
print(cv.best_params_)

In [81]:
random_forest = RandomForestRegressor(n_estimators=20,random_state=30,max_depth=30,min_samples_leaf=2,min_samples_split=5)
random_forest.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=30, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=5, min_weight_fraction_leaf=0.0,
                      n_estimators=20, n_jobs=None, oob_score=False,
                      random_state=30, verbose=0, warm_start=False)

In [82]:
y_pred_RF=random_forest.predict(x_test)

In [83]:
random_forest.score(x_test,y_test)

0.9597526409048482

In [84]:
r2_score(y_test,y_pred_RF)

0.9597526409048482

In [85]:
y_pred_train_RF = random_forest.predict(x_train)

In [86]:
print('Training set score:',(random_forest.score(x_train, y_train)))

print('Test set score:',(random_forest.score(x_test, y_test)))

Training set score: 0.9661786762247093
Test set score: 0.9597526409048482


### `Decision Tree`

In [87]:
from sklearn.tree import DecisionTreeRegressor

In [88]:
model2= DecisionTreeRegressor()
model2.fit(x_train,y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [89]:
y_pred_DT = model2.predict(x_test)

In [90]:
model2.score(x_test,y_test)

0.9398447805637685

In [91]:
r2_score(y_test,y_pred_DT)

0.9398447805637685

In [92]:
y_pred_train = model2.predict(x_train)

In [93]:
print('Training set score:',(model2.score(x_train, y_train)))

print('Test set score:',(model2.score(x_test, y_test)))

Training set score: 0.999938917224223
Test set score: 0.9398447805637685


### `XGBoost`

In [94]:
import xgboost
from sklearn.metrics import explained_variance_score

# Various hyper-parameters to tune
xgb1 = xgboost.XGBRegressor()
parameters = {
              'learning_rate': [.05,.07,.1,.13,.15],
              'max_depth': [3,5,7,9],
              'min_child_weight': [1,3,5,7],
              'colsample_bytree': [.3,.5,0.7],
              'n_estimators': [200,300,500,700]}

cv_scheme = KFold(n_splits=4, shuffle=True)

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = cv_scheme,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(x_train,y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

In [95]:
xgb = xgboost.XGBRegressor(n_estimators=200, learning_rate=0.1,
                           colsample_bytree=0.7, max_depth=7, min_child_weight= 1)

In [96]:
xgb.fit(x_train,y_train)

[06:11:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=None, n_estimators=200,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [97]:
y_pred_XG = xgb.predict(x_test)

In [98]:
xgb.score(x_test,y_test)

0.9643987368720436

In [99]:
r2_score(y_test,y_pred_XG)

0.9643987368720436

In [100]:
print('Training set score:',(xgb.score(x_train, y_train)))

print('Test set score:',(xgb.score(x_test, y_test)))

Training set score: 0.9997013101199312
Test set score: 0.9643987368720436
